In [7]:
import sys
import math
import random
import collections
import datetime
import itertools
import functools

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import IPython
import sympy as sp
import sympy.physics.vector as spv
import sympy.physics.mechanics as spm
import scipy.constants as spc

import IPython.display as ipd
spv.init_vprinting()
%matplotlib inline

In [8]:
print( f"""
    Python version {sys.version}
    IPython version {IPython.__version__}
    Numpy version {np.__version__}
    mathplotlib version {matplotlib.__version__}
    Pandas version {pd.__version__}
    Seaborn version {sns.__version__}
    """
)


    Python version 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
    IPython version 7.27.0
    Numpy version 1.21.2
    mathplotlib version 3.4.3
    Pandas version 1.3.3
    Seaborn version 0.11.2
    


In [9]:
def directory(obj):
    return [ 
        str for str in dir(obj) 
#        if callable(getattr(obj, str)) & ~str.startswith('_')
        if not str.startswith('_')
    ]

In [10]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [11]:

HALF = sp.S.Half
PI = sp.pi
E = sp.exp
POSITIVEINFINITY = sp.S.Infinity

In [12]:
def reference_frame(
    frame:str, 
    x=r'\imath', y=r'\jmath', z=r'\mathbf k'
) -> spv.ReferenceFrame:
    return spv.ReferenceFrame(
        frame, latexs=(
            fr'\; {{}}^\mathcal {frame} \hat {x}', 
            fr'\;{{}}^\mathcal {frame} \hat {y}', 
            fr'\: {{}}^\mathcal {frame} \hat {{z}}'
        )
    )
def vector(F:spv.ReferenceFrame, rx, ry, rz=0) -> spv.Vector:
    return rx*F.x +  ry*F.y + rz*F.z

def vector_cos(
    F:spv.ReferenceFrame, magnitude, 
    anglex, angley, anglez=sp.pi/2
) -> spv.Vector:
    return ( magnitude*
            (  sp.cos(anglex)*F.x 
             + sp.cos(angley)*F.y 
             + sp.cos(anglez)*F.z
            )
    )

def vector_line(
    start:spv.Vector, finish:spv.Vector
) -> spv.Vector:
    return finish - start

def vector_line_eqn(
    F:spv.ReferenceFrame, start:spv.Vector, finish:spv.Vector, kappa
) -> spv.Vector:
    return start + vector_line(start, finish).normalize()*kappa

def angle_between_vectors(a:spv.Vector, b:spv.Vector):
    return sp.acos(a.dot(b)/a.magnitude()/b.magnitude())
